---
Wykład 2
---

Wykład na podstawie:
- http://docs.julialang.org/en/stable/manual
- materialow Piotra Gawrona (Instytut Informatyki Teoretycznej i Stosowanej PAN)

Zawartosc:

- Typy
- Metody
- Konstruktory
- Konwersja i promocja typów
- Moduły
- Obsługa błędów

---

In [1]:
versioninfo()

Julia Version 0.6.2
Commit d386e40c17 (2017-12-13 18:08 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2673 v3 @ 2.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.9.1 (ORCJIT, haswell)


## Typy
---
### Wstęp

- System typów w Julii jest dynamiczny, ale ma pewne cechy systemu statycznego.
- Można pisać programy w Julii nie zwracając zbytnio uwagi na typy.
- Typy dzielimy na:
  + **abstrakcyjne** - nie można tworzyć ich instancji,
  + i **konkretne** - nie można po nich dziedziczyć.
 
- Wszystkie byty w Julii są obiektami, które mają dobrze zdefiniowany typ.
- Typ jest znany podczas wykonania a nie kompilacji.
- Tylko wartości mają typy. Zmienne **NIE** mają typów. Zmienne to nazwy związane z wartościami.
- Typy mogą być parametryzowane innymi typami.

### Deklaracja typów
- Do anotowania typów używamy operatora `::`. Ten operator oznacza asercję, mówiącą, że wartość jest danego typu.

In [2]:
# Wyrażenie po lewej jest typu Int
(1+2)::Int

3

In [3]:
# Wyrażenie po lewej jest typu AbstractFloat?
(1+2)::AbstractFloat

LoadError: [91mTypeError: typeassert: expected AbstractFloat, got Int64[39m

Jeżeli doczepimy operator `::` do zmiennej, to deklarujemy, że życzymy sobie by zmienna była skonwertowana do danego typu.

In [4]:
let
    x::Int8 = 10
    typeof(x)
end

Int8

In [5]:
let
    x = 10
    typeof(x)
end

Int64

Takie pseudo-deklaracje zmiennych możemy używać w następujących wersjach
```julia
local x::Int8  #  declaracja zmiennej lokalnej
x::Int8 = 10   # przypisanie 
```
Takie deklaracje dotyczą całego zakresu zmiennej. **Nie działają one w zakresie globalnym**.

In [6]:
global www::Int64

LoadError: [91msyntax: type declarations on global variables are not yet supported[39m

In [1]:
# wartość zwaracana będzie konwertowana do Float64
function sinc(x)::Float64
    if x == 0
        return 1
    end
    return sin(pi*x)/(pi*x)
end

sinc (generic function with 1 method)

In [4]:
sinc(0)

1.0


### Typy abstrakcyjne

Nie mogą być instancjonowane. Służą one jako węzły w grafie typów.

Typy abstrakcyjne definiuje się w następujący sposób.

In [9]:
abstract type MyType end
abstract type MySubType <: MyType end
supertype(MyType) |> println
supertype(MySubType) |> println

Any
MyType


Typ `Any` jest super-typem wszystkich typów. Również swoim własnym.

In [10]:
supertype(Any)

Any

Tak możemy sprawdzić typy pochodne po danym.

In [11]:
subtypes(Integer)

4-element Array{Union{DataType, UnionAll},1}:
 BigInt  
 Bool    
 Signed  
 Unsigned

Hierarchia typów numerycznych abstrakcyjnych w Julii.

```julia
abstract Number
abstract Real     <: Number
abstract AbstractFloat <: Real
abstract Integer  <: Real
abstract Signed   <: Integer
abstract Unsigned <: Integer
```

Napis `A <: B` oznacza `A` jest podtypem `B`.
Służy on do 
- definiowania bezpośredniego typu nadrzędnego
- lub sprawdzania czy typ `B` jest typem nadrzędnym `A`.

In [12]:
Integer <: Number

true

In [13]:
Integer <: AbstractFloat

false

### Primitive types

To typy, które składają się z bitów.

Deklaracja typów numerycznych w Julii wygląda tak (ilość bitów musi być podzielna przez 8):

```julia
primitive type «name» «bits» end
primitive type «name» <: «supertype» «bits» end
```

Przykłady:

```julia
primitive type Float16 <: AbstractFloat 16 end
primitive type Float32 <: AbstractFloat 32 end
primitive type Float64 <: AbstractFloat 64 end

primitive type Bool <: Integer 8 end
primitive type Char 32 end

primitive type Int8    <: Signed   8 end
primitive type UInt8   <: Unsigned 8 end
primitive type Int16   <: Signed   16 end
primitive type UInt16  <: Unsigned 16 end
primitive type Int32   <: Signed   32 end
primitive type UInt32  <: Unsigned 32 end
primitive type Int64   <: Signed   64 end
primitive type UInt64  <: Unsigned 64 end
primitive type Int128  <: Signed   128 end
primitive type UInt128 <: Unsigned 128 end
```

- Nazwa typu ma kluczowe znaczenie. 
- Typy `Bool` i `Int8` są reprezentowane tak samo ale nie można rzutować jednego na drugi automatycznie.

### Typy złożone niezmiennicze

Typy złożone to typy podobne do struktur w C. Każda instancja typu niezmienniczego jest powiązana z określonymi wartościami pola - same wartości pól mówią wszystko o obiekcie. 

In [14]:
struct Foo
    bar     # Domyślnie typ Any
    toto::Int
    baz::Float64
end

In [15]:
foo = Foo("Murphy", 42, 1.17)

Foo("Murphy", 42, 1.17)

In [16]:
typeof(foo)

Foo

In [17]:
foo2 = Foo((1,1),1,1.0)

Foo((1, 1), 1, 1.0)

In [18]:
typeof(foo2)

Foo

Do każdego typu tworzone są dwa konstruktory domyślne:
1. pierwszy akceptuje dowolne argumenty i wywołuje funkcję `convert()` na nich konwertując je do typów pól;
2. drugi akceptuje tylko wartości, które pasują dokładnie do typów pól.

In [19]:
# Typ ASCIIString nie jest konwertowalny do Int64
Foo("Murphy","Fish",1.0)

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Int64
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.[39m

In [20]:
# możemy zobaczyć pola zmiennej
fieldnames(foo)

3-element Array{Symbol,1}:
 :bar 
 :toto
 :baz 

In [21]:
# możemy odczytywać pola obiektu
foo.bar

"Murphy"

In [22]:
foo.toto

42

In [23]:
foo.baz

1.17

In [24]:
# Typu sa "immutable" tzn Nie możemy  zmieniać pola obiektu:
foo.bar = 1//2

LoadError: [91mtype Foo is immutable[39m


- Identyczne obiekty typu niezmienniczego są tymi samymi obiektami.

In [26]:
struct Pair
    a
    b
end

In [27]:
object_id(Pair(1,1)) |> println
object_id(Pair(1,1)) |> println

11589320051560747122
11589320051560747122


In [28]:
Pair(1,1)===Pair(1,1)

true

In [29]:
# Typy niezmiennicze mogą zawierać pola przypisane do obiektów zmiennych
x = Pair([1,2,3],[4,5,6])
x.a=1

LoadError: [91mtype Pair is immutable[39m

In [30]:
# ale:
x.a[1]=42

42

In [31]:
x.a, x.b

([42, 2, 3], [4, 5, 6])

In [32]:
x.a = [1,2,3]

LoadError: [91mtype Pair is immutable[39m

### Typy zmiennicze (mutable)

Obiekty typu zmienniczego są zwykle przydzielane do sterty i mają stabilne adresy pamięci.  Obiekt typu  zmienniczego  jest jak mały pojemnik, który może posiadać różne wartości w czasie, a zatem może być niezawodnie identyfikowany tylko z jego adresem. Natomiast instancja typu niezmienniczego jest powiązana z określonymi wartościami pola - same wartości pól mówią wszystko o obiekcie. Dlatego o tym, czy typ powinien byc niezmienniczy, powinno decydowac to, czy dwie instancje z tymi samymi wartościami pola beda uważane za identyczne, czy też mogą wymagać niezależnej zmiany w czasie. Jeśli zostaną uznane za identyczne, typ powinien prawdopodobnie być niezmienny.

Podsumowując, dwie zasadnicze właściwości definiują niezmienność w Julii:
    - obiekt typu niezmienniczego  jest przekazywany  przez kopiowanie, 
    - obiekt typu zmienniczego  jest przekazywany przez referencje.
    - niedozwolone jest modyfikowanie pól złożonego typu niezmienniczego.


In [33]:
mutable struct Bar
           baz
           qux::Float64
end

bar = Bar("Hello", 1.5);

In [34]:
bar.qux = 2.0

2.0

In [35]:
bar.baz = 1//2

1//2

In [36]:
bar2 = Bar("Hello", 1.5);

In [37]:
object_id(bar) |> println
object_id(bar2) |> println
bar === bar2

4917249588184458613
6217900026125947634


false

Typy złożone, które nie mają pól sa singletonami.

In [1]:
mutable struct NoFields
end

NoFields()===NoFields() 

true

### Wewnetrzna reprezentacja typów:

In [38]:
# wewnetrznie typy są reprezentowane jako DataType
typeof(Real)

DataType

In [39]:
fieldnames(DataType)

16-element Array{Symbol,1}:
 :name                     
 :super                    
 :parameters               
 :types                    
 :instance                 
 :layout                   
 :size                     
 :ninitialized             
 :uid                      
 :abstract                 
 :mutable                  
 Symbol("llvm::StructType")
 Symbol("llvm::DIType")    
 :depth                    
 :hasfreetypevars          
 :isleaftype               

In [40]:
Bar.mutable

true

In [41]:
Pair.mutable

false

### Sumy mnogościowe typów

In [42]:
IntOrString = Union{Int,AbstractString}

Union{AbstractString, Int64}

In [43]:
42 :: IntOrString

42

In [44]:
"Hello!" :: IntOrString

"Hello!"

In [45]:
1.0 :: IntOrString

LoadError: [91mTypeError: typeassert: expected Union{AbstractString, Int64}, got Float64[39m

### Typy parametryczne

Wszystkie typy typu `DataType` mogą być mieć parametry przez to tworząc rodzaj meta-typów.

Będziemy mówić o różnych rodzajach  typów parametrycznych:
- parametryczne typy złożone,
- parametryczne typy abstrakcyjne,
- typy Tuple (krotki)
- parametryczne typy bitowe.

#### Parametryczne typy złożone

In [5]:
struct Point{T}
    x::T
    y::T
end

In [47]:
Point{Float64}

Point{Float64}

In [48]:
Point{AbstractString}

Point{AbstractString}

In [49]:
Point{Point{Int}}

Point{Point{Int64}}

In [17]:
# Typ parametryczny też jest porządnym typem
Point

Point

In [51]:
# Jest on typem nadrzędnym wobec wszystkich wszystykich typów 
# konkretnych
Point{Float64} <: Point

true

In [52]:
Point{AbstractString} <: Point

true

In [53]:
# To oczywiście nie ma zbytnio sensu
Float64 <: Point

false

In [54]:
Float64 <: Point{Float64}

false

In [55]:
# Typy konkretne o różnych wartościach T 
# nie są wobec siebie typami podrzędnymi
# Parametryczne typy złożone w Julii sa inwariantne (typy zlozone nie zachowuja relacji typow swoich parametrow)
Point{Float64} <: Point{Int64}

false

In [56]:
Point{Int64} <: Point{Float64}

false

In [57]:
# UWAGA!
Float64 <: Real

true

In [58]:
# ALE!
Point{Float64} <: Point{Real}

false

Wynika to z tego, że `Point{Float64}` to po prostu dwie wartości `Float64` jedna po drugiej, a `Point{Real}` to para wskaźników. To kwestia wydajności.

Zatem taka metoda nie jest ciekawa, nie zadziala dla obiektow typu Point{Float64}:

In [59]:
function norm(p::Point{Real})
   sqrt(p.x^2 + p.y^2)
end

norm (generic function with 1 method)

In [60]:
# Natomiast ta będzie zachować porządnie
function norm{T<:Real}(p::Point{T})
   sqrt(p.x^2 + p.y^2)
end

norm (generic function with 2 methods)

In [61]:
#Albo:
function norm(p::Point{<:Real})
   sqrt(p.x^2 + p.y^2)
end

norm (generic function with 2 methods)

In [62]:
Albo:
function norm(p::Point{T} where T<:Real)
   sqrt(p.x^2 + p.y^2)
end 

LoadError: [91msyntax: line break in ":" expression[39m

 Obiekt typu `Point{Float64}` możemy skonstruować w następujący sposób:

In [63]:
p = Point{Float64}(1.0,2.0)

Point{Float64}(1.0, 2.0)

In [64]:
typeof(p)

Point{Float64}

Dla typów parametrycznych jest tworzony jeden konstruktor domyślny.

Często przy konstrukcji typu parametrycznego złożonego nie trzeba podawać parametru, gdyż może on być wyliczony na podstawie typów wartości zmiennych.

In [65]:
Point(1.0,2.0)

Point{Float64}(1.0, 2.0)

In [66]:
Point(1,2)

Point{Int64}(1, 2)

In [67]:
# To oczywiście nie działa
Point(1,2.5)
# Ale można jawnie stworzyć odpowiedni konstruktor *(patrz sekcja kontruktory poniżej)

LoadError: [91mMethodError: no method matching Point(::Int64, ::Float64)[0m
Closest candidates are:
  Point(::T, [91m::T[39m) where T at In[46]:2[39m

#### Parametryczne typy abstrakcyjne

In [13]:
# Tworzenie
abstract type Pointy{T} end

In [14]:
# Każdy typ Pointy{T} jest odrębnym typem dla każdego T bedącego typem lub liczbą typu integer. To drugie przydatne jest np. # przy rozroznianiu tablic o różnej ilości elementów za pomocą typów.
Pointy{Int64} <: Pointy

true

In [15]:
Pointy{1} <: Pointy

true

In [69]:
# Relacja dziedzicznia
Pointy{Int64} <: Pointy

true

In [70]:
# to nie jest prawda, parametryczne typy abstrakcyjne są rowniez inwariantne
Pointy{Float64} <: Pointy{Real}

false

In [71]:
Pointy{Real} <: Pointy{Float64}

false

In [72]:
# otrzymujemy typ, który zachowuje się jak kowariantny 
#(kowariantny=typy zlozone zachowuja relacje taka sama jak typy #parametrow ) :
# technicznie definiujemy zbiory typów (AllUnion types)
Pointy{Float64} <: Pointy{<:Real}

true

In [73]:
#tutaj otrzymujemy typ, który zachowuje się jak kontawariantny 
#(kontrawariantny=typy zlozone zachowuja relacje odwrotną w  stosunku do typów parametrow )
# technicznie definiujemy zbiory typów (AllUnion types)
Pointy{Real} <: Pointy{>:Float64}

true

In [74]:
# abstrakcyjne typy parametryczne slużą do tworzenia hierarchi typów
struct Point2D{T} <: Pointy{T}
           x::T
           y::T
       end

In [75]:
# Relacja herarchii jest zachowana
Point2D{Float64} <: Pointy{Float64}

true

In [76]:
Point2D{Real} <: Pointy{Real}

true

In [77]:
Point2D{AbstractString} <: Pointy{AbstractString}

true

In [78]:
# ta relacja jest także inwariantna
Point2D{Float64} <: Pointy{Real}

false

In [79]:
Point2D{Float64} <: Pointy{<:Real}

true

In [80]:
# Jesli potrzebujemy definicji Point1D, to zarówno Point2D {Float64}, 
# jak i Point1D {Float64} są implementacjami abstrakcji Pointy {Float64} 
# i podobnie dla każdego innego możliwego wyboru typu T. 
# Pozwala to na programowanie wspólnego interfejsu współdzielonego 
# przez wszystkie obiekty Pointy, zaimplementowanego zarówno dla Point1D i Point2D.
struct Point1D{T} <: Pointy{T}
           x::T
end

In [81]:
#Przy tworzeniu typu abstrakcyjnego  możemy od razu ograniczyć zakres dopuszczalnych parametrów
abstract type PointyReal{T<:Real} end

In [82]:
PointyReal{Float64}

PointyReal{Float64}

In [83]:
PointyReal{AbstractString}

LoadError: [91mTypeError: PointyReal: in T, expected T<:Real, got Type{AbstractString}[39m

In [84]:
# W taki sam sposób można ograniczać również zakres dla konkretnych typów złożonych
struct MyPointReal{T<:Real} <: Pointy{T}
    x::T
    y::T
end

In [85]:
# występujący w Julii przykład użycia systemu typów  dla typu Rational
struct Rational{T<:Integer} <: Real
    num::T
    den::T
end

#### Typy Tuple (krotki)

In [86]:
# Typy tuple są podobne do parametrycznych typów złożonych, ale:
# 1) mogą mieć dowolną liczbę parametrów.
# 2) są kowariantne w swoich parametrach: Tuple {Int} jest podtypem 
#   Tuple {Any}, dlatego też Tuple {Any} jest uważany za typ abstrakcyjny, 
#   a typy krotek są  konkretne, jeśli ich parametry są konkretne.
# 3) Krotki nie mają nazw pól; pola są dostępne tylko przez indeks.
struct Tuple2{A,B}
    a::A
    b::B
end

In [87]:
typeof((1,"foo",2.5))

Tuple{Int64,String,Float64}

In [88]:
# zmienna liczba elementów krotki
mytupletype = Tuple{AbstractString,Vararg{Int}}

Tuple{AbstractString,Vararg{Int64,N} where N}

In [89]:
isa(("1",), mytupletype)

true

In [90]:
isa(("1",1,2), mytupletype)

true

In [91]:
isa(("1",1,2,3.0), mytupletype)

false

#### Parametryczne typy bitowe

```julia
# 32-bit system:
primitive type Ptr{T} 32 end

# 64-bit system:
primitive type Ptr{T} 64 end
```

In [92]:
Ptr{Float64} <: Ptr

true

In [93]:
Ptr{Int64} <: Ptr

true

### UnionAll Types

 Typ parametryczny jest  nadrzędny wobec wszystkich wszystykich typów  konkretnych. 
 Wewnetrznie takie typy są reprezentowane jako UnionAll. Jest to iterowalna unia wszystkich typów
 dla wszystkich wartosci parametrów.

In [94]:
struct MyPoint{T}
    x::T
    y::T
end
typeof(MyPoint)

UnionAll

In [95]:
fieldnames(typeof(MyPoint))

2-element Array{Symbol,1}:
 :var 
 :body

In [96]:
MyPoint.var

T

In [97]:
typeof(MyPoint.var)

TypeVar

In [98]:
fieldnames(TypeVar)

3-element Array{Symbol,1}:
 :name
 :lb  
 :ub  

In [99]:
MyPoint.var.name

:T

In [100]:
# wszystkie typy sa nadtypami typu Union{}
MyPoint.var.lb

Union{}

In [101]:
#wszyskie typy sa podtypami typu Any
MyPoint.var.ub

Any

In [102]:
MyPoint.body

MyPoint{T}

In [103]:
typeof(MyPoint.body)

DataType

In [104]:
typeof(MyPoint{Int})

DataType

Zapis 
```julia
Ptr
```
jest skrótem dla 

```julia
Ptr{T}, where T```

co oznacza wszystkie wartości, których typem jest Ptr {T} dla pewnej wartości T. W tym kontekście parametr T jest często nazywany "zmienną typu", ponieważ działa jak zmienna dla typów. 

Możliwe jest  wiele parametrów, na przykład 
```julia
Array{T, N}, where N, where T
```

Zapis ograniczenia typów:
```julia
Array{<:Integer}
```

to skrót od:
```julia
Array{T} where T<:Integer
```

In [105]:
# ograniczenia mogą być od góry i od dołu
MyPoint{T} where Int<:T<:Number

MyPoint{T} where Int64<:T<:Number

In [106]:
typeof(MyPoint{T} where Int<:T<:Number)

UnionAll

In [107]:
(MyPoint{T} where Int<:T<:Number).var.name

:T

In [108]:
(MyPoint{T} where Int<:T<:Number).var.ub

Number

In [109]:
(MyPoint{T} where Int<:T<:Number).var.lb

Int64

In [110]:
(MyPoint{T} where Int<:T<:Number).body

MyPoint{Int64<:T<:Number}

### Aliasowanie
```julia
if Int === Int64
    const UInt = UInt64
else
    const UInt = UInt32
end```

### Operacje na typach

In [111]:
isa(1, Int)

true

In [112]:
typeof(Rational{Int})

DataType

In [113]:
supertype(Float64)

AbstractFloat

## Metody

- Metody definiują zachowanie funkcji dla danego zestawu typów jej parametrów.
- Użycie wszystkich typów parametrów funkcji do wyboru metody nazywa się mechanizmem multimetody  (_ang. multiple dispatch_). Jest to **najważniejsza** idea stojąca za językiem Julia.

In [114]:
# Funkcja zdefiniowana dla typów jej parametrów
f(x::Float64, y::Float64) = 2x + y

f (generic function with 1 method)

In [115]:
f(2.0, 3.0)

7.0

In [116]:
f(2.0, 3)

LoadError: [91mMethodError: no method matching f(::Float64, ::Int64)[0m
Closest candidates are:
  f(::Float64, [91m::Float64[39m) at In[114]:2[39m

In [117]:
f(Float32(2.0), 3.0)

LoadError: [91mMethodError: no method matching f(::Float32, ::Float64)[0m
Closest candidates are:
  f([91m::Float64[39m, ::Float64) at In[114]:2[39m

In [118]:
f(x::Number, y::Number) = 2000x + y

f (generic function with 2 methods)

In [119]:
f |> methods

# 2 methods for generic function "f":
f(x::Float64, y::Float64) in Main at In[114]:2
f(x::Number, y::Number) in Main at In[118]:1

In [120]:
f(2.0, 3)

4003.0

In [121]:
f(Float32(2.0), 3.0)

4003.0

In [122]:
# metoda, która wyłapuje wszystkie inne typy parametrów
f(x,y) = println("So long and thanks for all the fish")

f (generic function with 3 methods)

In [123]:
f("dolphin")

LoadError: [91mMethodError: no method matching f(::String)[0m
Closest candidates are:
  f(::Any, [91m::Any[39m) at In[122]:2
  f([91m::Float64[39m, [91m::Float64[39m) at In[114]:2
  f([91m::Number[39m, [91m::Number[39m) at In[118]:1[39m

In [124]:
f("dolphin", 42)

So long and thanks for all the fish


In [125]:
f(Float64, f)

So long and thanks for all the fish


In [126]:
which(f, (Float64, Float64))

f(x::Float64, y::Float64) in Main at In[114]:2

In [127]:
which(f, (Any, AbstractString))

f(x, y) in Main at In[122]:2

In [128]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

In [129]:
g(x::Float64, y) = 2x + y

g (generic function with 1 method)

In [130]:
g(x, y::Float64) = x + 2000y

g (generic function with 2 methods)

In [131]:
g(2.0, 3)

7.0

In [132]:
g(2, 3.0)

6002.0

In [133]:
g(2.0, 3.0)

LoadError: [91mMethodError: g(::Float64, ::Float64) is ambiguous. Candidates:
  g(x, y::Float64) in Main at In[130]:1
  g(x::Float64, y) in Main at In[129]:1
Possible fix, define
  g(::Float64, ::Float64)[39m

In [134]:
# Należy unikać takich sytuacji poprzez zdefiniowanie metody
g(x::Float64, y::Float64) = 2x + 2y

g (generic function with 3 methods)

In [135]:
g(2.0, 3.0)

10.0

### Metody parametryczne

In [136]:
# Sprawdzamy czy zmienne są tego samego typu
same_type(x::T, y::T) where {T} = true
same_type(x,y) = false

same_type (generic function with 2 methods)

In [137]:
same_type(1, 2)

true

In [138]:
same_type("foo", 2.0)

false

In [139]:
# Ograniczenie zakresu typów parametrów
same_type_numeric(x::T, y::T) where {T<:Number} = true
same_type_numeric(x::Number, y::Number) = false

same_type_numeric (generic function with 2 methods)

In [140]:
same_type_numeric(1, 2)

true

In [141]:
same_type_numeric(1, 2.0)

false

In [142]:
same_type_numeric("foo", 2.0)

LoadError: [91mMethodError: no method matching same_type_numeric(::String, ::Float64)[0m
Closest candidates are:
  same_type_numeric([91m::T<:Number[39m, ::T<:Number) where T<:Number at In[139]:2
  same_type_numeric([91m::Number[39m, ::Number) at In[139]:3[39m

In [143]:
same_type_numeric(Int32(1), Int64(2))

false

In [144]:
#parametry typów mogą dotyczyć jednocześnie argumentów funkcji, jak i typów tych argumentów np.  
myappend(v::Vector{T}, x::T) where {T} = [v..., x]

myappend (generic function with 1 method)

In [145]:
myappend([1,2,3],4)

4-element Array{Int64,1}:
 1
 2
 3
 4

In [146]:
myappend([1,2,3],2.5)

LoadError: [91mMethodError: no method matching myappend(::Array{Int64,1}, ::Float64)[0m
Closest candidates are:
  myappend(::Array{T,1}, [91m::T[39m) where T at In[144]:2[39m

In [147]:
myappend([1.0,2.0,3.0],4.0)

4-element Array{Float64,1}:
 1.0
 2.0
 3.0
 4.0

In [148]:
 myappend([1.0,2.0,3.0],4)

LoadError: [91mMethodError: no method matching myappend(::Array{Float64,1}, ::Int64)[0m
Closest candidates are:
  myappend(::Array{T,1}, [91m::T[39m) where T at In[144]:2[39m

In [149]:
#tutaj parametr typu jest zwracany poprzez metodę
mytypeof(x::T) where {T} = T

mytypeof (generic function with 1 method)

In [150]:
mytypeof(1)

Int64

In [151]:
mytypeof(1.0)

Float64

Przykład parametryzowanej metody o zmiennej liczbie argumentów. Użycie parametru N wymusza, aby liczba  indeksów była  równa rozmiarowi tablicy 
```julia
function getindex(A::AbstractArray{T,N}, indexes::Vararg{Number,N}) where {T,N}
```

### Typy 'podobne' do funkcji

In [152]:
struct Polynomial{R}
           coeffs::Vector{R}
       end

In [153]:
# Nie podajemy nazwy, tylko typ
function (p::Polynomial)(x)
           v = p.coeffs[end]
           for i = (length(p.coeffs)-1):-1:1
               v = v*x + p.coeffs[i]
           end
           return v
       end

In [154]:
 p = Polynomial([1,10,100])

Polynomial{Int64}([1, 10, 100])

In [155]:
p(3)

931

## Konstruktory

In [156]:
type Fooz
  bar
  baz
end

In [157]:
# Konstruktor domyślny
fooz = Fooz(1,2)

Fooz(1, 2)

### Konstruktory zewnętrzne
- Mogą tylko wywoływać inne konstruktory.

In [158]:
Fooz(x) = Fooz(x,x)

Fooz

In [159]:
Fooz(1)

Fooz(1, 1)

In [160]:
Fooz() = Fooz(0)

Fooz

In [161]:
Fooz()

Fooz(0, 0)

### Konstruktory wewnętrzne
- Konstruktory zewnętrzne pozwalają na:
 1. wymuszenie własności typów,
 2. konstrukcję typów, które odnoszą się same do siebie.
- Są one deklarowane wewnątrz bloku typu oraz
- mają dostęp do funkcji `new()`.
- Jeżeli konstruktor wewnętrzny został zdefiniowany, nie zostają utworzone automatycznie konstruktory domyślne.

In [162]:
struct OrderedPair
           x::Real
           y::Real
           OrderedPair(x,y) = x > y ? error("out of order") : new(x,y)
end

In [163]:
OrderedPair(1,2)

OrderedPair(1, 2)

In [164]:
OrderedPair(2,1)

LoadError: [91mout of order[39m

In [165]:
# Konstruktory domyślne. Te dwa typy są równoważne.
struct T1
  x::Int64
end

struct T2
  x::Int64
  T2(x) = new(x)
end

Dobra praktyka:

- Nie tworzyć zbyt wielu konstruktorów wewnętrznych: tylko te, które jawnie przyjmują wszystkie argumenty i wymuszają podstawową weryfikacje błędów. 
- Konstruktory  dla wartości domyślnych, transformacji pomocniczych itp., powinny być zewnętrzzne  

### Konstruktory dla typów parametrycznych

In [166]:
struct Pointp{T<:Real}
           x::T
           y::T
       end

In [167]:
Pointp(1,2)

Pointp{Int64}(1, 2)

In [168]:
Pointp(1.0,2.5)

Pointp{Float64}(1.0, 2.5)

In [169]:
# Gdy typ jest wnioskowany z typów argumentów wywołania konstruktora, 
# to te typy  muszą być zgodne - w przeciwnym razie nie można określić T
Pointp(1,2.5)

LoadError: [91mMethodError: no method matching Pointp(::Int64, ::Float64)[0m
Closest candidates are:
  Pointp(::T<:Real, [91m::T<:Real[39m) where T<:Real at In[166]:2[39m

In [170]:
Pointp{Int64}(1, 2)

Pointp{Int64}(1, 2)

In [171]:
# Tutaj błąd przy konwersji 2.5 do Int64
Pointp{Int64}(1.0,2.5)

LoadError: [91mInexactError()[39m

In [172]:
Pointp{Float64}(1.0, 2.5)

Pointp{Float64}(1.0, 2.5)

In [173]:
# tutaj zachodzi konwersja
Pointp{Float64}(1,2)

Pointp{Float64}(1.0, 2.0)

Przy takiej definicji typu automatycznie generują sie konstruktory:
```julia
 struct Point{T<:Real}
           x::T
           y::T
# działa, gdy podamy typ explicite
           Point{T}(x,y) where {T<:Real} = new(x,y)
       end
# działa, gdy nie podamy typu
Point(x::T, y::T) where {T<:Real} = Point{T}(x,y)
```

In [174]:
# jeśli chcemy dodać konstruktor działający na argumentach różnych typów możemy to zrobić tak:
Pointp(x::Int64, y::Float64) = Pointp(convert(Float64,x),y)

Pointp

In [175]:
Pointp(1,2.5)

Pointp{Float64}(1.0, 2.5)

In [176]:
# to wciąż nie działa:
Pointp(1.5,2)

LoadError: [91mMethodError: no method matching Pointp(::Float64, ::Int64)[0m
Closest candidates are:
  Pointp(::T<:Real, [91m::T<:Real[39m) where T<:Real at In[166]:2[39m

In [177]:
# aby stworzyć bardziej ogólny konstruktor używamy funcji promote(), która sprowadza oba typy do wspólnego typu: 
Pointp(x::Real, y::Real) = Pointp(promote(x,y)...)

Pointp

In [178]:
Pointp(1.5,2)

Pointp{Float64}(1.5, 2.0)

## Konwersja i promocja
### Konwersja
W Julii nie ma ukrytej konwersji typów. Konwersja jest jawnie definiowana.

In [179]:
x = 12

12

In [180]:
typeof(x)

Int64

In [181]:
convert(UInt8, x)

0x0c

In [182]:
typeof(ans)

UInt8

In [183]:
convert(AbstractFloat, x)

12.0

In [184]:
typeof(ans)

Float64

In [185]:
a = Any[1 2 3; 4 5 6]

2×3 Array{Any,2}:
 1  2  3
 4  5  6

In [186]:
convert(Array{Float64}, a)

2×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0

In [187]:
# niemożliwa konwersja
convert(AbstractFloat, "foo")

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type AbstractFloat
This may have arisen from a call to the constructor AbstractFloat(...),
since type constructors fall back to convert methods.[39m

Definiowanie własnej konwersji
```julia
function convert(::Type{Bool}, x::Real) 
    x==0 ? false : x==1 ? true : throw(InexactError())
end
```


In [188]:
# Przykład użycia
convert(Bool, 1)

true

In [189]:
convert(Bool, 0)

false

In [190]:
convert(Bool,5)

LoadError: [91mInexactError()[39m

Przykład rzeczywistej implementacji funkcji convert() w języku Julia
```julia
convert(::Type{T}, z::Complex) where {T<:Real} =
    (imag(z) == 0 ? convert(T, real(z)) : throw(InexactError()))
```

### Promocja

Promocja występuje wtedy, gdy dwie wartości są promowane do typu "wyższego" typu wspólnego. Nie ma to nic wspólnego z hierachią typów.

In [191]:
promote(1, 2.5)

(1.0, 2.5)

In [192]:
promote(1, 2.5, 3)

(1.0, 2.5, 3.0)

In [193]:
promote(2, 3//4)

(2//1, 3//4)

In [194]:
promote(1, 2.5, 3, 3//4)

(1.0, 2.5, 3.0, 0.75)

In [195]:
promote(1.5, im)

(1.5 + 0.0im, 0.0 + 1.0im)

In [196]:
promote(1 + 2im, 3//4) |> typeof

Tuple{Complex{Rational{Int64}},Complex{Rational{Int64}}}

- Definiowanie własnych promocji 
```julia
promote_rule(::Type{Float64}, ::Type{Float32} ) = Float64
promote_rule(::Type{UInt8}, ::Type{Int8}) = Int
promote_rule(::Type{BigInt}, ::Type{Int8}) = BigInt
```


In [197]:
# Testowanie promocji
promote_type(Int8, UInt16)

Int64

promocje są zdefioniowane w pliku https://github.com/JuliaLang/julia/blob/master/base/promotion.jl

## Moduły
- Pozwalają na organizację kodu np. w biblioteki.

```julia
module MyModule # definicja modułu, uwaga! nie ma wcięcia
using Lib # będziemy używać modułu Lib

using BigLib: thing1, thing2 #to samo co" BigLib.thing1, BigLib.thing2

import Base.show # działa jak using ale dla jednej nazwy
# Jeżeli chcemy dodawać metody do funkcji, to musimy użyć import

importall OtherLib # importuj wszystkie nazwy z modułu

export MyType, foo # deklaracja eksportowanych nazw

struct MyType
    x
end

bar(x) = 2x
foo(a::MyType) = bar(a.x) + 1

show(io, a::MyType) = print(io, "MyType $(a.x)")

include("file1.jl") # wklej zawartość pliku tutaj
end

```

Weźmy taki moduł

```julia
module MyModule

export x, y

x() = "x"
y() = "y"
p() = "p"

end
```


| Polecenie 			| W zakresie pojawia się | Można rozszerzać metodami | 
| - | - | - | 
| using MyModule 			| Wszystkie exportowane nazwy (x oraz y), MyModule.x, MyModule.y oraz MyModule.p | MyModule.x, MyModule.y oraz MyModule.p | 
| using MyModule.x, MyModule.p 	| x oraz p |  | 
| using MyModule: x, p | x oraz p |  | 
| import MyModule | MyModule.x, MyModule.y oraz MyModule.p | MyModule.x, MyModule.y oraz MyModule.p | 
| import MyModule.x, MyModule.p | x oraz p | x oraz p | 
| import MyModule: x, p | x oraz p | x oraz p | 
| importall MyModule | All exported names (x oraz y) | 

## Obsługa błędów
### Wyjątki
- Gdy pojawia się błąd zostaje rzucony wyjątek.
- Zdefiniowany jest szereg wyjątków wbudowanych.


In [198]:
# Przykład pojawienia się wyjątku
sqrt(-1)

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [199]:
# Definicja własnego wyjątku
struct MyCustomException <: Exception end

In [200]:
# funkcja throw()
f(x) = x>=0 ? exp(-x) : throw(DomainError()) 
# uwaga na nawiasy rzucamy obiektem typu wyjątek, nie typem

f (generic function with 4 methods)

In [201]:
f(1)

0.36787944117144233

In [202]:
f(-1)

LoadError: DomainError:

In [203]:
# Wyjątki mogą mieć parametry
struct MyUndefVarError <: Exception
   var::Symbol
end
function Base.showerror(io::IO, e::MyUndefVarError)
    print(io, e.var, " not defined");
end

In [204]:
throw(MyUndefVarError(:x))

LoadError: [91mx not defined[39m

### Błędy
Funkcja `error()` wyrzuca `ErrorException` i kończy wykonanie danej funkcji.

In [205]:
# dodatkowe funkcje informacyjne
info("Hi"); 1+1 # nie wyrzuca wyjątku

INFO: Hi


2

In [206]:
warn("Hi!"); 1+1 # nie wyrzuca wyjątku

2

In [207]:
error("Hi!!!"); 1+1 # wyrzuca wyjątek ErrorException

LoadError: [91mHi!!![39m

#### Konstrukcja `try/catch`.

In [208]:
# przykład
f(x) = try
         sqrt(x)
       catch
         sqrt(complex(x, 0))
       end

f (generic function with 4 methods)

In [209]:
f(1)

1.0

In [210]:
f(-1)

0.0 + 1.0im

In [211]:
# przykład, kiedy wyjątek zapamiętany jest w zmiennej e, którą możemy manipulować:
sqrt_second(x) = try
        sqrt(x[2])
    catch e
    println(e)
        if isa(e, DomainError)
           sqrt(complex(x[2], 0))
        elseif isa(e, BoundsError)
           sqrt(x)
        end
    end

sqrt_second (generic function with 1 method)

In [212]:
sqrt_second([1 4])

2.0

In [213]:
sqrt_second([1 -4])

DomainError()


0.0 + 2.0im

In [214]:
sqrt_second(9)

BoundsError(#undef, #undef)


3.0

In [215]:
sqrt_second(-9)

BoundsError(#undef, #undef)


LoadError: DomainError:

#### Konstrukcja `finally`

```julia
f = open("file")
try
    # operate on file f
finally
    close(f)
end
```